In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [31]:
path_client_info = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt"
path_exp_web_data1 = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt"
path_exp_web_data2 = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt"
path_test_grouping = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt"

In [33]:
df_client_info = pd.read_csv(path_client_info)
df_client_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [35]:
#EDA for numerical variables
df_client_info.describe().round(2)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,70609.00,70595.00,70595.00,70594.00,70595.00,70595.00,70595.00,70595.00
mean,5004991.88,12.05,150.66,46.44,2.26,147445.24,3.38,5.57
std,2877277.63,6.87,82.09,15.59,0.53,301508.71,2.24,2.35
min,169.00,2.00,33.00,13.50,1.00,13789.42,0.00,1.00
25%,2519329.00,6.00,82.00,32.50,2.00,37346.84,1.00,4.00
50%,5016978.00,11.00,136.00,47.00,2.00,63332.90,3.00,5.00
75%,7483085.00,16.00,192.00,59.00,2.00,137544.90,6.00,7.00
max,9999839.00,62.00,749.00,96.00,8.00,16320040.15,7.00,9.00


In [37]:
#check only non-numerical variable
df_client_info['gendr'].value_counts()

gendr
U    24122
M    23724
F    22746
X        3
Name: count, dtype: int64

In [39]:
#tukeys_test logic invertically applied to select the IQR for defining the "Average Customer"
def tukeys_test_inliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    # Define bounds for the outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify the inliers
    inliers = data[(data > lower_bound) & (data < upper_bound)]
    
    return inliers

inliers_balance = tukeys_test_inliers(df_client_info['bal'])
inliers_balance
inliers_balance.mean() #e.g. 78119.33 mean balance of the IQR
#theoretically df could be cleaned by df_cleaned = df[~df.drop(inliers.index)]

78119.33428352268

In [41]:
#EDA (mean median std) on the most important categories for the IQR clients only:

important_cats = ['clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'bal', 'calls_6_mnth', 'logons_6_mnth']

iqr_mean_dict = {'mean':[], 'median':[], 'std': []}


for i in important_cats:
    inliers = tukeys_test_inliers(df_client_info[i])
    iqr_mean_dict['mean'] += [round(inliers.mean(),2)]
    iqr_mean_dict['median'] += [round(inliers.median(),2)]
    iqr_mean_dict['std'] += [round(inliers.std(),2)]

iqr_mean_dict
iqr_mean_Joe = pd.DataFrame(iqr_mean_dict)
iqr_mean_Joe['index'] = important_cats
iqr_mean_Joe = iqr_mean_Joe.set_index('index')
iqr_mean_Joe

,mean,median,std
index,,,
clnt_tenure_yr,11.72,11.00,6.33
clnt_tenure_mnth,145.38,132.00,73.89
clnt_age,46.44,47.00,15.59
bal,78119.33,55699.76,59622.11
calls_6_mnth,3.38,3.00,2.24
logons_6_mnth,5.57,5.00,2.35


In [43]:
df_client_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [45]:
#check second table, the experiment's web data table 1
df_exp_web_data1 = pd.read_csv(path_exp_web_data1)
df_exp_web_data1
#343141 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [46]:
#check third table, the experiment's web data table 2
df_exp_web_data2 = pd.read_csv(path_exp_web_data2)
df_exp_web_data2
#412264 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [47]:
#merge the two experiment tables via concat:
df_exp_web_data = pd.concat([df_exp_web_data1, df_exp_web_data2], axis=0)
df_exp_web_data
#755405 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [48]:
df_exp_web_data['client_id'].nunique()
#120k clients, why?

120157

In [49]:
#check fourth table of test grouping info per client
df_ab_test_grouping = pd.read_csv(path_test_grouping)
df_ab_test_grouping
#70609 rows -> matches with number of client_ids in df_client_info

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


In [55]:
#check for NaN values:
df_ab_test_grouping.isna().sum()
# 0 NaNs for client_id but 20109 for Variation -> 20k of 70k clients did not participate in the test?

client_id        0
Variation    20109
dtype: int64

In [63]:
#merge client_info and test_grouping_info to check if the two test group have different demographics
df_client_grouping_info = df_client_info.merge(df_ab_test_grouping, on='client_id', how='left')
df_client_grouping_info[df_client_grouping_info['Variation'] == 'Test'].describe().round(2)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,26968.00,26961.00,26961.00,26961.00,26961.00,26961.00,26961.00,26961.00
mean,4999386.45,11.98,149.85,47.16,2.25,148962.61,3.06,6.10
std,2877454.52,6.84,81.78,15.51,0.53,307373.71,2.19,2.18
min,555.00,2.00,33.00,17.00,1.00,23789.44,0.00,3.00
25%,2516165.25,6.00,82.00,33.50,2.00,39729.20,1.00,4.00
50%,5025620.50,11.00,134.00,47.50,2.00,65468.36,3.00,6.00
75%,7469166.75,15.00,191.00,59.50,2.00,139478.58,5.00,8.00
max,9999832.00,55.00,669.00,96.00,7.00,16320040.15,6.00,9.00


In [65]:
df_client_grouping_info[df_client_grouping_info['Variation'] == 'Control'].describe().round(2)
#good sampling, the descriptive stats are very similar

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,23532.00,23527.00,23527.00,23526.00,23527.00,23527.00,23527.00,23527.00
mean,5013963.49,12.09,151.06,47.50,2.26,150147.33,3.13,6.17
std,2877470.20,6.88,82.13,15.52,0.54,295806.95,2.18,2.17
min,1028.00,2.00,33.00,17.00,2.00,23789.61,0.00,3.00
25%,2515401.00,6.00,82.00,33.50,2.00,40123.35,1.00,4.00
50%,5023892.00,11.00,137.00,48.50,2.00,66024.18,3.00,6.00
75%,7483811.25,16.00,192.00,60.00,2.00,140606.04,5.00,8.00
max,9998921.00,55.00,669.00,96.00,7.00,8292996.21,6.00,9.00


In [45]:
#Further EDA: check relationship between client_id, visitor_id and visit_id in web_exp_ table:
df_exp_web_data_filtered

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
294346,1574008,117364417_77840596075,528720790_71583064618_169151,start,2017-05-06 23:43:27
294391,2908510,814969699_90652851448,562606085_36368381773_92090,start,2017-05-10 22:57:17
294392,2908510,814969699_90652851448,562606085_36368381773_92090,step_2,2017-05-10 22:56:31
294393,2908510,814969699_90652851448,562606085_36368381773_92090,step_1,2017-05-10 22:56:23


In [31]:
#check if client_id always has the same visitor_id:

# Group by 'client_id' and calculate the number of unique 'visitor_id' values
unique_visitor_counts = df_exp_web_data_filtered.groupby('client_id')['visitor_id'].nunique()

# Check if every 'client_id' has only 1 unique 'visitor_id'
consistent_mapping = unique_visitor_counts == 1

# Print any inconsistencies
if consistent_mapping.all():
    print("All client_ids always map to the same visitor_id.")
else:
    inconsistent_clients = unique_visitor_counts[unique_visitor_counts > 1]
    print("The following client_ids map to multiple visitor_ids:")
    print(inconsistent_clients)

#4971 (of 69657)  client ids with multiple visitor ids, visitor_id = device?!

The following client_ids map to multiple visitor_ids:
client_id
1680       2
4653       2
6130       3
7367       2
9282       2
          ..
9994607    2
9995309    2
9996088    2
9996952    2
9998342    2
Name: visitor_id, Length: 4971, dtype: int64


In [47]:
#merge info about grouping (test vs. control) of clients into web experiment data
df_web_exp = df_exp_web_data_filtered.merge(participation_df, on='client_id', how='left')
df_web_exp['Variation'].value_counts()
#Test       177847
#Control    143462

Variation
Test       177847
Control    143462
Name: count, dtype: int64

In [51]:
df_web_exp.info() #321309 entries
df_web_exp['visitor_id'].nunique() #56011 -> ~500 more than client_id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321309 entries, 0 to 321308
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     321309 non-null  int64 
 1   visitor_id    321309 non-null  object
 2   visit_id      321309 non-null  object
 3   process_step  321309 non-null  object
 4   date_time     321309 non-null  object
 5   Variation     321309 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.7+ MB


56011

In [57]:
#check for visitor_ids with multiple client ids:
#group by 'visitor_id'
grouped = df_web_exp.groupby('visitor_id')

#find groups where 'Variation' has more than one unique value
ambiguous_visitors = grouped.filter(lambda x: x['Variation'].nunique() > 1)

#get unique visitor IDs from confusing groups
ambiguous_visitor_ids = ambiguous_visitors['visitor_id'].unique()

#output the result
print(f"Visitor_id groups with more than one unique 'Variation' value: {len(ambiguous_visitor_ids)}")
#print(f"List of ambiguous visitor group IDs: {ambiguous_visitor_ids}")
#185 ambiguous visitor_ids

Visitor_id groups with more than one unique 'Variation' value: 185


In [61]:
#example check
df_web_exp[df_web_exp['visitor_id'] == '454881699_42713236223']

,client_id,visitor_id,visit_id,process_step,date_time,Variation
3599,7460349,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:14:01,Control
3600,7460349,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:13:30,Control
3601,7460349,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:12:55,Control
3602,7460349,454881699_42713236223,399371316_39189899496_559976,start,2017-04-05 16:12:36,Control
192188,7208557,454881699_42713236223,399371316_39189899496_559976,start,2017-04-05 16:19:36,Test
192189,7208557,454881699_42713236223,399371316_39189899496_559976,step_2,2017-04-05 16:15:33,Test
192190,7208557,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:15:03,Test
192191,7208557,454881699_42713236223,399371316_39189899496_559976,start,2017-04-05 16:14:55,Test
304877,7208557,454881699_42713236223,430342294_16750074009_383709,start,2017-05-01 18:24:00,Test
304878,7208557,454881699_42713236223,430342294_16750074009_383709,step_2,2017-05-01 18:20:31,Test


In [65]:
#as they have different variations but the same visitor_id (same device?) and even the same visit_id (sessions?) in some cases
#we have to get rid of these 185 visitor_ids -> data cleaning

<class 'pandas.core.frame.DataFrame'>
Index: 318872 entries, 0 to 321308
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     318872 non-null  int64 
 1   visitor_id    318872 non-null  object
 2   visit_id      318872 non-null  object
 3   process_step  318872 non-null  object
 4   date_time     318872 non-null  object
 5   Variation     318872 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.0+ MB
